In [12]:
pip install librosa

  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached numba-0.60.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached msgpack-1.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached llvmlite-0.43.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
Using cached librosa-0.11.0-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
Using cached msgpack-1.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (405 kB)
Using cached numba-0.60.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
Using cached llvmlite-0.43.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.

In [10]:
pip install matplotlib numpy torch torchaudio pandas transformers peft scikit-learn

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 6.9 MB/s  0:00:016m0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [peft]4/5 [peft]erate]
Note: you may need to restart the kernel to use updated packages.


In [13]:
"""
Simplified EAT + KMeans anomaly detection pipeline focusing on performance
"""
from __future__ import annotations

import os
from pathlib import Path
import sys, math, json, argparse, time, re, random
from typing import List, Optional, Tuple, Dict

# Headless plotting
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

import pandas as pd

from transformers import AutoModel

try:
    from peft import LoraConfig, get_peft_model, TaskType
    PEFT_AVAILABLE = True
except Exception:
    PEFT_AVAILABLE = False

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    silhouette_score, roc_curve, auc, precision_recall_curve, 
    f1_score, confusion_matrix, accuracy_score
)

# Load modules from existing script
sys.path.insert(0, '/home/sey87151/EAT_Clean')
from v3_reporter import EATFeatureExtractor

# LoRa defaults
DEFAULT_LORA_R = 32
DEFAULT_LORA_ALPHA = 64
DEFAULT_LORA_DROPOUT = 0.05
DEFAULT_FT_LR = 1e-4
DEFAULT_FT_EPOCHS = 50

In [15]:
def infer_label_from_path(p):
    """Infer label from path: 0=normal, 1=anomaly"""
    p_str = str(p).lower()
    if "anomaly" in p_str or "abnormal" in p_str:
        return 1
    else:
        return 0

def infer_domain_from_path(p):
    """Infer domain from path"""
    return Path(p).parent.name

def chunked(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def collect_domain_paths(root_dir, machine):
    """Collect train and test paths for a machine based on folder structure."""
    machine_dir = Path(root_dir) / machine
    if not machine_dir.exists():
        print(f"Machine directory not found: {machine_dir}")
        return [], []
    
    train_dir = machine_dir / "train"
    test_dir = machine_dir / "test"
    
    if not train_dir.exists() or not test_dir.exists():
        print(f"Train or test directory not found in: {machine_dir}")
        return [], []
    
    # Collect training paths (all from train folder - should be normal)
    train_paths = list(train_dir.glob("*.wav"))
    
    # Collect test paths (all from test folder - mix of normal and anomaly)
    test_paths = list(test_dir.glob("*.wav"))
    
    print(f"Found {len(train_paths)} training files")
    print(f"Found {len(test_paths)} test files")
    
    return train_paths, test_paths

# ----------------- LoRA utilities (simplified copy) -----------------
ATTN_TOKENS = [
    "qkv", "proj",
    "q_proj", "k_proj", "v_proj", "o_proj", "out_proj",
    "query", "key", "value", "in_proj", "in_proj_weight", "in_proj_bias",
]

def present_attention_tokens(model: nn.Module) -> List[str]:
    names = [n for n, _ in model.named_modules()]
    present = [tok for tok in ATTN_TOKENS if any(tok in n for n in names)]
    return sorted(set(present), key=present.index)

def resolve_lora_targets(model: nn.Module, override: Optional[str]) -> List[str]:
    names = [n for n, _ in model.named_modules()]
    if override:
        raw = [t.strip() for t in override.split(',') if t.strip()]
        filtered = [t for t in raw if any(t in n for n in names)]
        if filtered:
            return sorted(set(filtered), key=filtered.index)
        print(f"[LoRA][WARN] None of --lora_targets matched actual module names: {raw}")
    present = present_attention_tokens(model)
    if present:
        return present
    fallback = ["qkv", "q_proj", "k_proj", "v_proj"]
    fb_filtered = [t for t in fallback if any(t in n for n in names)]
    if fb_filtered:
        return fb_filtered
    raise ValueError("Could not auto-discover LoRA targets; pass --lora_targets explicitly.")

def find_lora_target_names(model: nn.Module, tokens: List[str], last_n: int,
                           include_only_attn: bool = True, include_mlp_fc1: bool = False) -> List[str]:
    block_ids = []
    for n, _ in model.named_modules():
        m = re.match(r"^model\.blocks\.(\d+)\.", n)
        if m:
            block_ids.append(int(m.group(1)))
    if not block_ids:
        return []
    cutoff = max(0, max(block_ids) - last_n + 1)
    selected: List[str] = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        m = re.match(r"^model\.blocks\.(\d+)\.", name)
        if not m:
            continue
        blk = int(m.group(1))
        if blk < cutoff:
            continue
        if include_only_attn and ".attn." not in name:
            if include_mlp_fc1 and name.endswith(".mlp.fc1"):
                pass
            else:
                continue
        if not any(tok in name for tok in tokens):
            continue
        selected.append(name)
    out=[]; seen=set()
    for n in selected:
        if n not in seen:
            out.append(n); seen.add(n)
    return out


# Dataset Management and Utility Functions

This section contains utility functions for:
- Path management and label inference
- Data collection from machine directories
- General helper functions for data processing

# LoRA (Low-Rank Adaptation) Utilities

This section implements utilities for LoRA fine-tuning:
- Attention module detection
- Target module resolution
- LoRA configuration helpers

In [16]:
# LoRA attention tokens and utility functions
ATTN_TOKENS = [
    "qkv", "proj",
    "q_proj", "k_proj", "v_proj", "o_proj", "out_proj",
    "query", "key", "value", "in_proj", "in_proj_weight", "in_proj_bias",
]

def present_attention_tokens(model: nn.Module) -> List[str]:
    names = [n for n, _ in model.named_modules()]
    present = [tok for tok in ATTN_TOKENS if any(tok in n for n in names)]
    return sorted(set(present), key=present.index)

def resolve_lora_targets(model: nn.Module, override: Optional[str]) -> List[str]:
    names = [n for n, _ in model.named_modules()]
    if override:
        raw = [t.strip() for t in override.split(',') if t.strip()]
        filtered = [t for t in raw if any(t in n for n in names)]
        if filtered:
            return sorted(set(filtered), key=filtered.index)
        print(f"[LoRA][WARN] None of --lora_targets matched actual module names: {raw}")
    present = present_attention_tokens(model)
    if present:
        return present
    fallback = ["qkv", "q_proj", "k_proj", "v_proj"]
    fb_filtered = [t for t in fallback if any(t in n for n in names)]
    if fb_filtered:
        return fb_filtered
    raise ValueError("Could not auto-discover LoRA targets; pass --lora_targets explicitly.")

def find_lora_target_names(model: nn.Module, tokens: List[str], last_n: int,
                           include_only_attn: bool = True, include_mlp_fc1: bool = False) -> List[str]:
    block_ids = []
    for n, _ in model.named_modules():
        m = re.match(r"^model\.blocks\.(\d+)\.", n)
        if m:
            block_ids.append(int(m.group(1)))
    if not block_ids:
        return []
    cutoff = max(0, max(block_ids) - last_n + 1)
    selected: List[str] = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        m = re.match(r"^model\.blocks\.(\d+)\.", name)
        if not m:
            continue
        blk = int(m.group(1))
        if blk < cutoff:
            continue
        if include_only_attn and ".attn." not in name:
            if include_mlp_fc1 and name.endswith(".mlp.fc1"):
                pass
            else:
                continue
        if not any(tok in name for tok in tokens):
            continue
        selected.append(name)
    out=[]; seen=set()
    for n in selected:
        if n not in seen:
            out.append(n); seen.add(n)
    return out

# Dataset and Neural Network Components

This section includes:
- Custom dataset class for pseudo-labeled data
- Neural network heads (Linear and CosFace)
- Data collation functions

In [17]:
class PseudoLabelDataset(torch.utils.data.Dataset):
    def __init__(self, wavs: List[np.ndarray], labels: np.ndarray, sr: int, target_len: Optional[int]):
        assert len(wavs) == len(labels)
        self.wavs = wavs
        self.labels = labels.astype(np.int64)
        self.sr = sr
        self.target_len = target_len
    def __len__(self):
        return len(self.wavs)
    def __getitem__(self, idx: int):
        w = self.wavs[idx]; y = int(self.labels[idx])
        w_t = torch.tensor(w, dtype=torch.float32)
        if w_t.ndim!=1:
            w_t = w_t.view(-1)
        w_t = w_t - w_t.mean()
        fb = torchaudio.compliance.kaldi.fbank(
            w_t.unsqueeze(0), htk_compat=True, sample_frequency=self.sr,
            use_energy=False, window_type="hanning", num_mel_bins=128,
            dither=0.0, frame_shift=10,
        )
        T_true = fb.size(0)
        if self.target_len is not None:
            if T_true < self.target_len:
                fb = F.pad(fb, (0,0,0,self.target_len - T_true))
            elif T_true > self.target_len:
                fb = fb[: self.target_len, :]
        fb = (fb + 4.288) / 4.469
        L = min(T_true, self.target_len or T_true)
        return fb.unsqueeze(0), y, L

def collate_fb(items):
    fbs, ys, lens = zip(*items)
    x = torch.stack(fbs, dim=0)
    y = torch.tensor(ys, dtype=torch.long)
    lengths = torch.tensor(lens, dtype=torch.long)
    return x, y, lengths

# Neural Network Heads and K-means Optimization

Implementation of different neural network heads and clustering optimization:

In [18]:
class LinearHead(nn.Module):
    def __init__(self, in_dim: int, n_classes: int):
        super().__init__(); self.fc = nn.Linear(in_dim, n_classes)
    def forward(self, x): return self.fc(x)

class CosFaceHead(nn.Module):
    def __init__(self, in_dim: int, n_classes: int, s: float = 64.0, m: float = 0.35):
        super().__init__()
        # W has shape [in_dim, n_classes]; columns are class weight vectors
        self.W = nn.Parameter(torch.randn(in_dim, n_classes))
        nn.init.xavier_uniform_(self.W)
        self.s = float(s)
        self.m = float(m)
    def forward(self, x, y):
        # L2-normalize features and class weights
        x_n = F.normalize(x, dim=1)
        W_n = F.normalize(self.W, dim=0)  # normalize columns (each class vector)
        # Cosine logits
        logits = x_n @ W_n  # [B, C]
        # Subtract margin from the target class cosine
        # (equivalent to logits.scatter_ with gathered target)
        onehot = F.one_hot(y, num_classes=logits.size(1)).float()
        logits_m = logits - self.m * onehot
        # Scale by s
        return self.s * logits_m

def auto_choose_k(feats: np.ndarray, k_grid: List[int] = None, seed: int = 42) -> int:
    if k_grid is None: k_grid = [12,18,24,32]
    optimal_k = k_grid[0]; best=-1.0
    X = StandardScaler().fit_transform(feats)
    for k in k_grid:
        try:
            km = KMeans(n_clusters=k, n_init="auto", random_state=seed)
            labels = km.fit_predict(X)
            if len(set(labels))==1: continue
            sil = silhouette_score(X, labels)
        except Exception:
            sil=-1.0
        if sil>best: best=sil; optimal_k=k
    return optimal_k

# Configuration and Parameters Setup

Set up experiment parameters and configure the anomaly detection pipeline:

In [35]:
# Configuration parameters for the experiment
class Config:
    def __init__(self):
        self.dataset_root = "./dataset"
        self.machine = "bearing"
        self.model_id = "worstchan/EAT-base_epoch30_pretrain"
        self.sr = 16000
        self.target_length = 1024
        self.batch_size = 32
        self.max_train = 10000
        self.seed = 42
        self.pool_mode = "mean_with_cls"
        self.use_l2_norm = False
        
        # LoRa fine-tuning parameters
        self.do_lora_ft = True
        self.auto_hparam = False
        self.lora_r = DEFAULT_LORA_R
        self.lora_alpha = DEFAULT_LORA_ALPHA
        self.lora_dropout = DEFAULT_LORA_DROPOUT
        self.lora_targets = None
        self.lora_last_n = 4
        self.lora_include_mlp_fc1 = False
        self.ft_lr = DEFAULT_FT_LR
        self.ft_epochs = DEFAULT_FT_EPOCHS
        self.grad_clip = 1.0
        self.ft_warmup = 0.05
        self.ft_val_split = 0.1
        self.early_stop_patience = 10
        self.use_cosface = False
        
        # CosFace hyperparameters
        self.cosface_s = 64.0
        self.cosface_m = 0.35
        
        # Embedding compactness & margin enhancements
        self.add_center_loss = False
        self.center_ema = 0.9
        self.lambda_center = 0.1
        self.add_supcon = False
        self.supcon_temp = 0.07
        self.lambda_supcon = 0.1

# Initialize configuration
args = Config()

# Set random seeds for reproducibility
torch.manual_seed(args.seed)
np.random.seed(args.seed)
random.seed(args.seed)

print(f"=== Simplified EAT Anomaly Detection for {args.machine} ===")
print(f"Model: {args.model_id}")
print(f"Batch size: {args.batch_size}")
print(f"Target length: {args.target_length}")
print(f"Sample rate: {args.sr}")
print(f"Random seed: {args.seed}")

=== Simplified EAT Anomaly Detection for bearing ===
Model: worstchan/EAT-base_epoch30_pretrain
Batch size: 32
Target length: 1024
Sample rate: 16000
Random seed: 42


# Data Collection and Loading

Collect training and testing data from the dataset directories:

In [36]:
# 1) Collect data from train and test folders
train_paths, test_paths = collect_domain_paths(args.dataset_root, args.machine)

if len(train_paths) == 0 or len(test_paths) == 0:
    print("Insufficient data for training/testing.")
    print(f"Found {len(train_paths)} training files and {len(test_paths)} test files")
else:
    # Create labels for test data based on filenames
    test_labels = []
    for path in test_paths:
        test_labels.append(infer_label_from_path(path))
    test_labels = np.array(test_labels)
    
    # Count normal and anomaly in test set
    n_test_normal = np.sum(test_labels == 0)
    n_test_anomaly = np.sum(test_labels == 1)

    print(f"Training on {len(train_paths)} normal samples")
    print(f"Testing on {len(test_paths)} samples ({n_test_normal} normal + {n_test_anomaly} anomalous)")
    
    # Display sample paths
    print(f"\nSample training paths:")
    for i, path in enumerate(train_paths[:3]):
        print(f"  {i+1}. {path}")
    
    print(f"\nSample test paths:")
    for i, path in enumerate(test_paths[:3]):
        label = "anomaly" if test_labels[i] == 1 else "normal"
        print(f"  {i+1}. {path} -> {label}")

Found 1100 training files
Found 200 test files
Training on 1100 normal samples
Testing on 200 samples (100 normal + 100 anomalous)

Sample training paths:
  1. dataset/bearing/train/section_00_source_train_normal_0364_noAttribute.wav
  2. dataset/bearing/train/section_00_source_train_normal_0643_noAttribute.wav
  3. dataset/bearing/train/section_00_source_train_normal_0188_noAttribute.wav

Sample test paths:
  1. dataset/bearing/test/section_00_target_test_normal_0020_noAttribute.wav -> normal
  2. dataset/bearing/test/section_00_target_test_anomaly_0031_noAttribute.wav -> anomaly
  3. dataset/bearing/test/section_00_source_test_anomaly_0043_noAttribute.wav -> anomaly


# Feature Extraction - Training Data

Extract features from training audio files using the EAT model:

In [37]:
# 2) Load audio and extract features
feat = EATFeatureExtractor(model_id=args.model_id, sr=args.sr,
                           target_length=args.target_length, pool_mode=args.pool_mode)

# Train features
print("Extracting training features...")
wavs_tr = []
for p in train_paths:
    try:
        wav, _ = torchaudio.load(p)
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        wavs_tr.append(wav.squeeze().numpy())
    except Exception as e:
        print(f"Error loading {p}: {e}")
        continue

if len(wavs_tr) > args.max_train:
    idx = np.random.choice(len(wavs_tr), size=args.max_train, replace=False)
    wavs_tr = [wavs_tr[i] for i in idx]
    print(f"Randomly selected {len(wavs_tr)} training samples from {len(train_paths)} total")

feats_tr = []
tot = math.ceil(len(wavs_tr) / args.batch_size)
for bi, batch in enumerate(chunked(wavs_tr, args.batch_size), 1):
    cls = feat.encode_batch(batch)
    feats_tr.append(cls)
    if bi % 10 == 0 or bi == tot:
        print(f"[train] {bi}/{tot} -> {cls.shape}")

X_train = np.vstack(feats_tr) if feats_tr else np.zeros((0, 768), dtype=np.float32)
print(f"Training features: {X_train.shape}")

Extracting training features...


/home/sey87151/EAT_Clean/.venv/lib64/python3.9/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/sey87151/EAT_Clean/.venv/lib64/python3.9/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issu

[train] 10/35 -> (32, 768)
[train] 20/35 -> (32, 768)
[train] 20/35 -> (32, 768)
[train] 30/35 -> (32, 768)
[train] 30/35 -> (32, 768)
[train] 35/35 -> (12, 768)
Training features: (1100, 768)
[train] 35/35 -> (12, 768)
Training features: (1100, 768)


# Feature Extraction - Test Data

Extract features from test audio files:

In [38]:
# Test features
print("Extracting test features...")
wavs_te = []
for p in test_paths:
    try:
        wav, _ = torchaudio.load(p)
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        wavs_te.append(wav.squeeze().numpy())
    except Exception as e:
        print(f"Error loading {p}: {e}")
        continue

feats_te = []
tot_te = math.ceil(len(wavs_te) / args.batch_size)
for bi, batch in enumerate(chunked(wavs_te, args.batch_size), 1):
    cls = feat.encode_batch(batch)
    feats_te.append(cls)
    if bi % 10 == 0 or bi == tot_te:
        print(f"[test] {bi}/{tot_te} -> {cls.shape}")

X_test = np.vstack(feats_te) if feats_te else np.zeros((0, 768), dtype=np.float32)
print(f"Test features: {X_test.shape}")

# Verify feature dimensions match expectations
print(f"\nFeature extraction summary:")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Test samples: {X_test.shape[0]}")
print(f"  Feature dimensions: {X_train.shape[1] if len(X_train) > 0 else 'N/A'}")
print(f"  Normal test samples: {n_test_normal}")
print(f"  Anomalous test samples: {n_test_anomaly}")

Extracting test features...
[test] 7/7 -> (8, 768)
Test features: (200, 768)

Feature extraction summary:
  Training samples: 1100
  Test samples: 200
  Feature dimensions: 768
  Normal test samples: 100
  Anomalous test samples: 100
[test] 7/7 -> (8, 768)
Test features: (200, 768)

Feature extraction summary:
  Training samples: 1100
  Test samples: 200
  Feature dimensions: 768
  Normal test samples: 100
  Anomalous test samples: 100


# Feature Processing and Preprocessing

Apply normalization, standardization, and dimensionality reduction:

In [60]:
# 3) Feature processing
print("\n=== FEATURE PROCESSING ===")

# Apply L2 normalization if requested
if args.use_l2_norm:
    print("Applying L2 normalization...")
    X_train = X_train / (np.linalg.norm(X_train, axis=1, keepdims=True) + 1e-6)
    X_test = X_test / (np.linalg.norm(X_test, axis=1, keepdims=True) + 1e-6)

# Standardization
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

print(f"Standardization applied - Training mean: {X_train_std.mean():.6f}, std: {X_train_std.std():.6f}")
print(f"Standardization applied - Test mean: {X_test_std.mean():.6f}, std: {X_test_std.std():.6f}")

# # PCA for dimensionality reduction
# pca = PCA(n_components=0.95, random_state=args.seed)
# X_train_pca = pca.fit_transform(X_train_std)
# X_test_pca = pca.transform(X_test_std)

# print(f"After PCA: {X_train_pca.shape[1]} dimensions (explained variance: {pca.explained_variance_ratio_.sum():.3f})")
# print(f"Original dimensions: {X_train_std.shape[1]} -> Reduced to: {X_train_pca.shape[1]}")
# print(f"Variance explained by top 5 components: {pca.explained_variance_ratio_[:5]}")


=== FEATURE PROCESSING ===
Standardization applied - Training mean: -0.000000, std: 1.000000
Standardization applied - Test mean: -0.003322, std: 1.029913


# Clustering Analysis and Optimal K Selection

Find the optimal number of clusters using silhouette analysis:

In [61]:
# 4) Find optimal number of clusters
print("\n=== CLUSTERING ===")

K_range = range(8, min(64, len(X_train)//10), 4)
silhouette_scores = []

print(f"Testing K values from {min(K_range)} to {max(K_range)}...")

for k in K_range:
    kmeans_test = KMeans(n_clusters=k, random_state=args.seed, n_init=10)
    labels = kmeans_test.fit_predict(X_train)
    
    if len(np.unique(labels)) > 1:
        sil_score = silhouette_score(X_train, labels)
        silhouette_scores.append(sil_score)
    else:
        silhouette_scores.append(-1)
    
    print(f"K={k}: silhouette={silhouette_scores[-1]:.4f}")

# Choose best K
best_idx = np.argmax(silhouette_scores)
optimal_k = list(K_range)[best_idx]
best_sil = silhouette_scores[best_idx]

print(f"\nOptimal K: {optimal_k} (silhouette: {best_sil:.4f})")

# Train final model
km = KMeans(n_clusters=optimal_k, random_state=args.seed, n_init=10)
km.fit(X_train)

print(f"Final KMeans model trained with {optimal_k} clusters")
print(f"Cluster sizes: {np.bincount(km.labels_)}")
print(f"Cluster centers shape: {km.cluster_centers_.shape}")


=== CLUSTERING ===
Testing K values from 8 to 60...
K=8: silhouette=0.1435
K=8: silhouette=0.1435
K=12: silhouette=0.1325
K=12: silhouette=0.1325
K=16: silhouette=0.1259
K=16: silhouette=0.1259
K=20: silhouette=0.1286
K=20: silhouette=0.1286
K=24: silhouette=0.1227
K=24: silhouette=0.1227
K=28: silhouette=0.1288
K=28: silhouette=0.1288
K=32: silhouette=0.1295
K=32: silhouette=0.1295
K=36: silhouette=0.1320
K=36: silhouette=0.1320
K=40: silhouette=0.1394
K=40: silhouette=0.1394
K=44: silhouette=0.1397
K=44: silhouette=0.1397
K=48: silhouette=0.1487
K=48: silhouette=0.1487
K=52: silhouette=0.1469
K=52: silhouette=0.1469
K=56: silhouette=0.1517
K=56: silhouette=0.1517
K=60: silhouette=0.1468

Optimal K: 56 (silhouette: 0.1517)
K=60: silhouette=0.1468

Optimal K: 56 (silhouette: 0.1517)
Final KMeans model trained with 56 clusters
Cluster sizes: [16 34 24 17  7 42 33 16  6 29 15  6 23 18 20 39  6 13  6 18 34 13 21 16
 20 24  6 29 25 13 30 21 20 25 13  7 37 33 19 18 27 21 12 12 25 16 29 24


# Anomaly Scoring with Mahalanobis Distance

Implement per-cluster Mahalanobis distance for anomaly scoring:

In [62]:
# 5) Score test data (Per-cluster Mahalanobis distance instead of Euclidean)
print("\n=== ANOMALY SCORING (Mahalanobis) ===")

from scipy.linalg import cho_factor, cho_solve

mus = km.cluster_centers_
labels_train = km.predict(X_train)

covs = []
cho_info = []
lam = 1e-3  # ridge for numerical stability
feat_dim = X_train.shape[1]

print(f"Computing covariance matrices for {optimal_k} clusters...")

for k in range(optimal_k):
    Xk = X_train[labels_train == k]
    print(f"  Cluster {k}: {len(Xk)} samples")
    
    # if len(Xk) < feat_dim + 2:
    #     # Fallback: isotropic covariance if too few points
    #     if Xk.shape[0] > 1:
    #         Sk = np.cov(Xk.T)
    #     else:
    #         Sk = np.eye(feat_dim)
    #     print(f"    Using fallback covariance (too few samples)")
    # else:
    Sk = np.cov(Xk.T)
    
    # Regularize
    Sk = Sk + lam * np.eye(Sk.shape[0])
    try:
        c, lower = cho_factor(Sk, overwrite_a=False, check_finite=False)
    except Exception as e:
        # In rare singular cases, bump ridge and retry once
        print(f"    Cholesky failed, increasing regularization")
        Sk = Sk + 10 * lam * np.eye(Sk.shape[0])
        c, lower = cho_factor(Sk, overwrite_a=False, check_finite=False)
    
    covs.append(Sk)
    cho_info.append((c, lower))

def maha_min(X, mus, cho_info):
    """Compute minimum Mahalanobis distance across all clusters"""
    dists = []
    for x in X:
        ds = []
        for mu, (c, lower) in zip(mus, cho_info):
            r = x - mu
            q = cho_solve((c, lower), r, check_finite=False)
            ds.append(np.sqrt(np.dot(r, q)))
        dists.append(min(ds))
    return np.asarray(dists)

print(f"Computing anomaly scores for {len(X_test_pca)} test samples...")
scores = maha_min(X_test, mus, cho_info)

print(f"Anomaly scores computed:")
print(f"  Min score: {scores.min():.4f}")
print(f"  Max score: {scores.max():.4f}")
print(f"  Mean score: {scores.mean():.4f}")
print(f"  Std score: {scores.std():.4f}")


=== ANOMALY SCORING (Mahalanobis) ===
Computing covariance matrices for 56 clusters...
  Cluster 0: 16 samples
  Cluster 1: 34 samples
  Cluster 2: 24 samples
  Cluster 3: 17 samples
  Cluster 4: 7 samples
  Cluster 5: 42 samples
  Cluster 6: 33 samples
  Cluster 7: 16 samples
  Cluster 8: 6 samples
  Cluster 9: 29 samples
  Cluster 10: 15 samples
  Cluster 11: 6 samples
  Cluster 12: 23 samples
  Cluster 13: 18 samples
  Cluster 14: 20 samples
  Cluster 15: 39 samples
  Cluster 16: 6 samples
  Cluster 17: 13 samples
  Cluster 18: 6 samples
  Cluster 19: 18 samples
  Cluster 20: 34 samples
  Cluster 21: 13 samples
  Cluster 22: 21 samples
  Cluster 23: 16 samples
  Cluster 24: 20 samples
  Cluster 25: 24 samples
  Cluster 26: 6 samples
  Cluster 27: 29 samples
  Cluster 28: 25 samples
  Cluster 29: 13 samples
  Cluster 30: 30 samples
  Cluster 31: 21 samples
  Cluster 32: 20 samples
  Cluster 33: 25 samples
  Cluster 34: 13 samples
  Cluster 35: 7 samples
  Cluster 36: 37 samples
  Cl

# Performance Evaluation and Metrics

Evaluate the anomaly detection performance using various metrics:

In [65]:
# 6) Evaluate performance
print("\n=== EVALUATION ===")

# ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, scores)
auc_score = auc(fpr, tpr)

# pAUC (partial AUC for FPR <= 0.1)
max_fpr = 0.1
fpr_partial = fpr[fpr <= max_fpr]
tpr_partial = tpr[fpr <= max_fpr]
if len(fpr_partial) > 1:
    pauc = auc(fpr_partial, tpr_partial) / max_fpr
else:
    pauc = 0.0

print(f"AUC: {auc_score:.4f}")
print(f"pAUC (FPR ≤ 0.1): {pauc:.4f}")

# Initialize results dictionary for metrics
results = {
    "machine": args.machine,
    "auc": float(auc_score),
    "pauc": float(pauc),
    "optimal_k": int(optimal_k),
    "silhouette_score": float(best_sil),
    "n_features": int(X_train_pca.shape[1]),
    "explained_variance": float(pca.explained_variance_ratio_.sum()),
    "n_train": len(X_train_pca),
    "n_test": len(X_test_pca)
}

# Score distribution analysis
normal_scores = scores[test_labels == 0]
anomaly_scores = scores[test_labels == 1]

print(f"\nScore distribution analysis:")
print(f"  Normal samples - Mean: {normal_scores.mean():.4f}, Std: {normal_scores.std():.4f}")
print(f"  Anomaly samples - Mean: {anomaly_scores.mean():.4f}, Std: {anomaly_scores.std():.4f}")
print(f"  Separation (anomaly_mean - normal_mean): {anomaly_scores.mean() - normal_scores.mean():.4f}")

# Find optimal threshold using F1 score
precision, recall, pr_thresholds = precision_recall_curve(test_labels, scores)
f1_scores = []
threshold_range = np.linspace(scores.min(), scores.max(), 100)

for thresh in threshold_range:
    y_pred = (scores >= thresh).astype(int)
    if len(np.unique(y_pred)) > 1:
        f1 = f1_score(test_labels, y_pred)
        f1_scores.append(f1)
    else:
        f1_scores.append(0)

optimal_thresh_idx = np.argmax(f1_scores)
optimal_threshold = threshold_range[optimal_thresh_idx]
optimal_f1 = f1_scores[optimal_thresh_idx]

y_pred_optimal = (scores >= optimal_threshold).astype(int)
cm = confusion_matrix(test_labels, y_pred_optimal)

print(f"\nOptimal threshold analysis:")
print(f"  Optimal threshold: {optimal_threshold:.4f}")
print(f"  F1 score at optimal threshold: {optimal_f1:.4f}")
print(f"  Accuracy at optimal threshold: {accuracy_score(test_labels, y_pred_optimal):.4f}")
print(f"  Confusion matrix:")
print(f" {cm}")

# Update results with additional metrics
additional_results = {
    "optimal_threshold": float(optimal_threshold),
    "optimal_f1": float(optimal_f1),
    "accuracy_at_optimal": float(accuracy_score(test_labels, y_pred_optimal)),
    "confusion_matrix": cm.tolist(),
    "normal_score_mean": float(np.mean(normal_scores)),
    "normal_score_std": float(np.std(normal_scores)),
    "anomaly_score_mean": float(np.mean(anomaly_scores)),
    "anomaly_score_std": float(np.std(anomaly_scores)),
    "n_test_normal": int(np.sum(test_labels == 0)),
    "n_test_anomaly": int(np.sum(test_labels == 1))
}
results.update(additional_results)


=== EVALUATION ===
AUC: 0.6242
pAUC (FPR ≤ 0.1): 0.2340

Score distribution analysis:
  Normal samples - Mean: 6.9157, Std: 1.0213
  Anomaly samples - Mean: 7.6279, Std: 1.4640
  Separation (anomaly_mean - normal_mean): 0.7122

Optimal threshold analysis:
  Optimal threshold: 5.4012
  F1 score at optimal threshold: 0.6803
  Accuracy at optimal threshold: 0.5300
  Confusion matrix:
 [[  6  94]
 [  0 100]]


# Visualization and Plotting

Generate comprehensive plots for analysis and reporting:

In [48]:
# 7) Generate comprehensive plots
print("\n=== GENERATING PLOTS ===")

results_dir = Path(f"./results_{args.machine}")
results_dir.mkdir(exist_ok=True)
plots_dir = results_dir / "plots"
plots_dir.mkdir(exist_ok=True)


=== GENERATING PLOTS ===


# Plot 1: ROC Curve with AUC and pAUC

In [49]:
# Plot 1: ROC Curve with AUC and pAUC
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')

# Highlight pAUC region
fpr_pauc = fpr[fpr <= max_fpr]
tpr_pauc = tpr[fpr <= max_fpr]
plt.fill_between(fpr_pauc, 0, tpr_pauc, alpha=0.3, color='red', 
                 label=f'pAUC ≤ {max_fpr} (pAUC = {pauc:.4f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'ROC Curve - {args.machine.capitalize()} Anomaly Detection', fontsize=14)
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)

# Add performance text box
textstr = f'Clusters: {optimal_k}\\nSilhouette: {best_sil:.3f}\\nFeatures: {X_train_pca.shape[1]}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.65, 0.15, textstr, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig(plots_dir / "roc_curve.png", dpi=300, bbox_inches='tight')
plt.show()
print("ROC curve plot saved")

ROC curve plot saved


# Plot 2: Score Distribution Analysis

In [50]:
# Plot 2: Score Distribution
plt.figure(figsize=(12, 6))

# Subplot 1: Histogram of scores by class
plt.subplot(1, 2, 1)
normal_scores = scores[test_labels == 0]
anomaly_scores = scores[test_labels == 1]

plt.hist(normal_scores, bins=30, alpha=0.7, label=f'Normal (n={len(normal_scores)})', 
         color='blue', density=True)
plt.hist(anomaly_scores, bins=30, alpha=0.7, label=f'Anomaly (n={len(anomaly_scores)})', 
         color='red', density=True)
plt.xlabel('Anomaly Score (Distance to Nearest Cluster)', fontsize=11)
plt.ylabel('Density', fontsize=11)
plt.title('Score Distribution by Class', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Box plot
plt.subplot(1, 2, 2)
data_to_plot = [normal_scores, anomaly_scores]
labels_box = ['Normal', 'Anomaly']
box_plot = plt.boxplot(data_to_plot, tick_labels=labels_box, patch_artist=True)
box_plot['boxes'][0].set_facecolor('lightblue')
box_plot['boxes'][1].set_facecolor('lightcoral')
plt.ylabel('Anomaly Score', fontsize=11)
plt.title('Score Distribution (Box Plot)', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(plots_dir / "score_distribution.png", dpi=300, bbox_inches='tight')
plt.show()
print("Score distribution plot saved")

Score distribution plot saved


# Plot 3: Threshold Analysis

In [51]:
# Plot 3: Threshold Analysis
plt.figure(figsize=(12, 8))

# Subplot 1: Precision-Recall curve
plt.subplot(2, 2, 1)
plt.plot(recall, precision, linewidth=2, label=f'PR Curve')
plt.xlabel('Recall', fontsize=11)
plt.ylabel('Precision', fontsize=11)
plt.title('Precision-Recall Curve', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: F1 vs Threshold
plt.subplot(2, 2, 2)
plt.plot(threshold_range, f1_scores, linewidth=2, color='green')
plt.axvline(optimal_threshold, color='red', linestyle='--', 
            label=f'Optimal Threshold = {optimal_threshold:.3f}')
plt.xlabel('Threshold', fontsize=11)
plt.ylabel('F1 Score', fontsize=11)
plt.title(f'F1 Score vs Threshold (Max F1 = {optimal_f1:.3f})', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: TPR and FPR vs Threshold
plt.subplot(2, 2, 3)
if len(thresholds) > 1:
    plt.plot(thresholds[1:], tpr[1:], linewidth=2, label='True Positive Rate', color='blue')
    plt.plot(thresholds[1:], fpr[1:], linewidth=2, label='False Positive Rate', color='red')
    plt.axvline(optimal_threshold, color='green', linestyle='--', alpha=0.7)
plt.xlabel('Threshold', fontsize=11)
plt.ylabel('Rate', fontsize=11)
plt.title('TPR and FPR vs Threshold', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: Performance metrics summary
plt.subplot(2, 2, 4)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix\\n(Optimal Threshold)', fontsize=12)
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Anomaly'])
plt.yticks(tick_marks, ['Normal', 'Anomaly'])

# Add text annotations
thresh_cm = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh_cm else "black")

plt.ylabel('True Label', fontsize=11)
plt.xlabel('Predicted Label', fontsize=11)

plt.tight_layout()
plt.savefig(plots_dir / "threshold_analysis.png", dpi=300, bbox_inches='tight')
plt.show()
print("Threshold analysis plot saved")

Threshold analysis plot saved


# Plot 4: Clustering Visualization (t-SNE)

In [53]:
# Plot 4: Clustering Visualization (if enough data points)
if len(X_train) > 50:
    plt.figure(figsize=(10, 8))
    try:
        from sklearn.manifold import TSNE
        # Combine train and test for consistent visualization
        X_combined = np.vstack([X_train, X_test])
        
        print("Computing t-SNE for visualization...")
        tsne = TSNE(n_components=2, random_state=args.seed, perplexity=min(30, len(X_combined)//4))
        X_2d = tsne.fit_transform(X_combined)
        
        # Separate back into train/test
        X_train_2d = X_2d[:len(X_train)]
        X_test_2d = X_2d[len(X_train):]

        # Plot training data points
        plt.scatter(X_train_2d[:, 0], X_train_2d[:, 1], c='lightblue', alpha=0.6, 
                   s=20, label=f'Training (Normal, n={len(X_train_2d)})')
        
        # Plot test data points
        normal_mask = test_labels == 0
        anomaly_mask = test_labels == 1
        
        plt.scatter(X_test_2d[normal_mask, 0], X_test_2d[normal_mask, 1], 
                   c='blue', alpha=0.8, s=30, label=f'Test Normal (n={np.sum(normal_mask)})')
        plt.scatter(X_test_2d[anomaly_mask, 0], X_test_2d[anomaly_mask, 1], 
                   c='red', alpha=0.8, s=30, label=f'Test Anomaly (n={np.sum(anomaly_mask)})')
        
        plt.xlabel('t-SNE Component 1', fontsize=12)
        plt.ylabel('t-SNE Component 2', fontsize=12)
        plt.title(f'Clustering Visualization (t-SNE) - {args.machine.capitalize()}', fontsize=14)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(plots_dir / "clustering_visualization.png", dpi=300, bbox_inches='tight')
        plt.show()
        print("Clustering visualization (t-SNE) plot saved")
    except Exception as e:
        print(f"Could not generate t-SNE visualization: {e}")
else:
    print("Not enough data points for t-SNE visualization")

Computing t-SNE for visualization...
Clustering visualization (t-SNE) plot saved
Clustering visualization (t-SNE) plot saved


# Results Summary and Export

Final summary and save results to files:

In [56]:
# Final results summary and export
print(f"\n=== FINAL SUMMARY ===")
print(f"Plots saved to {plots_dir}")
print(f"  - ROC curve with pAUC highlighting")
print(f"  - Score distribution analysis")
print(f"  - Threshold optimization analysis")
if len(X_train) > 50:
    print(f"  - Clustering visualization (t-SNE)")

# Feature and clustering stats
print(f"\nCluster statistics:")
print(f"  Number of clusters: {optimal_k}")
print(f"  Silhouette score: {best_sil:.4f}")
print(f"  Feature dimensions: {X_train.shape[1]}")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")

print(f"\nPerformance Summary:")
print(f"  AUC: {auc_score:.4f}")
print(f"  pAUC: {pauc:.4f}")
print(f"  Optimal F1: {optimal_f1:.4f}")
print(f"  Accuracy: {accuracy_score(test_labels, y_pred_optimal):.4f}")

# Save results
results_dir.mkdir(exist_ok=True)
with open(results_dir / "results_with_plots.json", "w") as f:
    json.dump(results, f, indent=2)

np.save(results_dir / "test_scores.npy", scores)
np.save(results_dir / "test_labels.npy", test_labels)

print(f"\nResults saved to {results_dir}")
print(f"  - results_with_plots.json: Complete metrics")
print(f"  - test_scores.npy: Anomaly scores")
print(f"  - test_labels.npy: True labels")
print("\n✅ Anomaly detection pipeline completed successfully!")


=== FINAL SUMMARY ===
Plots saved to results_bearing/plots
  - ROC curve with pAUC highlighting
  - Score distribution analysis
  - Threshold optimization analysis
  - Clustering visualization (t-SNE)

Cluster statistics:
  Number of clusters: 56
  Silhouette score: 0.1517
  Feature dimensions: 768
  Training samples: 1100
  Test samples: 200

Performance Summary:
  AUC: 0.6242
  pAUC: 0.2340
  Optimal F1: 0.6803
  Accuracy: 0.5300

Results saved to results_bearing
  - results_with_plots.json: Complete metrics
  - test_scores.npy: Anomaly scores
  - test_labels.npy: True labels

✅ Anomaly detection pipeline completed successfully!

Results saved to results_bearing
  - results_with_plots.json: Complete metrics
  - test_scores.npy: Anomaly scores
  - test_labels.npy: True labels

✅ Anomaly detection pipeline completed successfully!


In [34]:
class PseudoLabelDataset(torch.utils.data.Dataset):
    def __init__(self, wavs: List[np.ndarray], labels: np.ndarray, sr: int, target_len: Optional[int]):
        assert len(wavs) == len(labels)
        self.wavs = wavs
        self.labels = labels.astype(np.int64)
        self.sr = sr
        self.target_len = target_len
    def __len__(self):
        return len(self.wavs)
    def __getitem__(self, idx: int):
        w = self.wavs[idx]; y = int(self.labels[idx])
        w_t = torch.tensor(w, dtype=torch.float32)
        if w_t.ndim!=1:
            w_t = w_t.view(-1)
        w_t = w_t - w_t.mean()
        fb = torchaudio.compliance.kaldi.fbank(
            w_t.unsqueeze(0), htk_compat=True, sample_frequency=self.sr,
            use_energy=False, window_type="hanning", num_mel_bins=128,
            dither=0.0, frame_shift=10,
        )
        T_true = fb.size(0)
        if self.target_len is not None:
            if T_true < self.target_len:
                fb = F.pad(fb, (0,0,0,self.target_len - T_true))
            elif T_true > self.target_len:
                fb = fb[: self.target_len, :]
        fb = (fb + 4.288) / 4.469
        L = min(T_true, self.target_len or T_true)
        return fb.unsqueeze(0), y, L

def collate_fb(items):
    fbs, ys, lens = zip(*items)
    x = torch.stack(fbs, dim=0)
    y = torch.tensor(ys, dtype=torch.long)
    lengths = torch.tensor(lens, dtype=torch.long)
    return x, y, lengths